# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

In [52]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/A_Testing_of_Colors.jpg/800px-A_Testing_of_Colors.jpg
!wget https://www.reflexiones.life/wp-content/uploads/2019/12/persona-mala.jpg

--2020-06-28 00:42:00--  https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/A_Testing_of_Colors.jpg/800px-A_Testing_of_Colors.jpg
Resolviendo upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Conectando con upload.wikimedia.org (upload.wikimedia.org)[208.80.154.240]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 297453 (290K) [image/jpeg]
Grabando a: “800px-A_Testing_of_Colors.jpg”

800px-A_Testing_of_ 100%[===================>] 290,48K   419KB/s    en 0,7s    

2020-06-28 00:42:01 (419 KB/s) - “800px-A_Testing_of_Colors.jpg” guardado [297453/297453]

--2020-06-28 00:42:01--  https://www.reflexiones.life/wp-content/uploads/2019/12/persona-mala.jpg
Resolviendo www.reflexiones.life (www.reflexiones.life)... 184.171.242.18
Conectando con www.reflexiones.life (www.reflexiones.life)[184.171.242.18]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 39437 (39K) [image/jpeg]
Grabando a: “p

In [1]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

Using TensorFlow backend.


In [30]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("content/La_noche_estrellada1.jpg")
result_prefix = Path("content/output")
iterations = 100


style_reference_image_path = Path("content/800px-A_Testing_of_Colors.jpg")
base_image_path = Path("content/persona-mala.jpg")



# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:\
**style_weight** -> la ponderacion de la contribucion de del estilo en la loss (el Beta en el paper se divide en la cantidad de capas que se toman en este caso 10/5 capas)\
**content_weight** -> la ponderacion de la contribucion del contenido en la loss (el Alfa del paper)\
**total_variation_weight** -> peso de la contribucion de la tercer componente de la loss que penaliza una gran varianza en la imagen que se genera. Busca mantener la imagen es localmente consistente.

In [13]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [1]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

NameError: name 'load_img' is not defined

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:\
Esta funcion recibe una imagen y devuelve un array. La anteultima linea agega una primer dimension y la ultima linea, segun la documentacion de tensorflow convierte de RGB a BGR y quita la media (zero-centered) respecto de ImageNet.

In [5]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:\
Esta celda toma el array y genera una imagen nuevamente RGB y revierte el centrado sumando la media, anteriormente lo hizo con la media de ImageNet, ahora lo hace nuevamente. Esta funcion revierte el efecto de la declarada en la celda anterior.

In [6]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [8]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [9]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [10]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?\
Es el producto interno de las respuestas de los filtros en una capa (correlacion entre filtros). Se usa como la textura que representa el estilo que quiero dar a la imagen final. Se computa para los filtros de la pintura y la imagen generada para formar la loss de estilo en una capa, finalmente se suman las loss de cada capa y se ponderan por el style_weight/cant_capas 
- ¿Por qué se permutan las dimensiones de x?\
La matriz Gram es el producto interno de las respuestas de los filtros, entre ellos. Permuto para que la primer dimension sean los filtros (Filtros o channels, filas, columnas) y que quede bien al trasponer. Por defecto los filtros estan al final (filas, columnas, filtros o channels)

In [11]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:\
**style_loss** -> Es la distancia en la representacion del estilo entre las activaciones de los filtros para la pintura (referencia de estilo) y la imagen que estoy creando. Se hace con las matrices Gram de n capas. Al minimizar la distancia, la textura de la imagen que genero sera mas parecida a la textura de la pintura.\
**content_loss** -> Es la distancia en la activacion de los filtros para la foto y para la imagen que se esta generando. Cuando se minimiza esta distancia la imagen que genero se acerca a la foto.\
**total_variation_loss** -> Computa variacion en horizontal y vertical, usando pequeños desplazamientos en la imagen que genero. De esta forma puedo penalizar cambios abruptos y lograr que la imagen sea localmente coerente.

In [20]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [21]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [22]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:             ,  :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:,  :img_nrows - 1, 1:             , :])
    return K.sum(K.pow(a + b, 1.25))


In [23]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [36]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:\
**eval_loss_and_grads** -> Genera una salida con los valores de la loss y los gradientes que obtine de los objetos que ya genere en las celdas anteriores.\
**Evaluator** -> Esta clase es para pasar los parametros a la funcion scipy.optimize.fmin_l_bfgs_b que recibe los valores de la loss y los gradientes por separado. Evaluator hace uso de eval_loss_and_grads que con un solo procesamiento devuelve loss y gradientes, luego Evaluator puede entregarlos por separado para fmin_l_bfgs_b.\
**Generacion de imagenes** -> En esta ultima celda se corren 100 iteraciones ejecutando la minimizacion de la loss (cada iteracion ejecuta 20 aproximaciones) cada iteracion se guarda como una nueva imagen.

-Diferencia con la implementacion del paper:\
En el paper solo se usan las loss de style y content, no la total_loss. De hecho observo imagenes con mayor nitidez con ponderaciones bajas de total_loss.

-Alternativa:\
Como alternativa, tensorflow ofrece ofrece la posibilidad de usar Adam o SGD en lugar de la aproximacion con la funcion fmin_l_bfgs_b.\
Ademas otra publicacion
>Ulyanov, D., Lebedev, V., Vedaldi, A., and Lempitsky, V. S. (2016). Texture networks: Feed-forwardsynthesis of textures and stylized images.  InProceedings of the 33nd International Conferenceon Machine Learning, ICML 2016, New York City, NY, USA, June 19-24, 2016, pages 1349–1357

[Perceptual Losses for Real-Time Style Transfer and Super-Resolution](https://cs.stanford.edu/people/jcjohns/eccv16/) ,
https://cs.stanford.edu/people/jcjohns/papers/eccv16/JohnsonECCV16.pdf \
ofrece un metdo para pre entrenar la red que aplicara los estilos para luego obtener transformaciones mucho mas rapidas.



In [ ]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

Observacion:
Se observa un impacto fuerte del estilo desde la primer iteracion, desde la 10 a la 20 se ven pocas modificaciones en la forma de la "pincelada" y los cambios mas notorios son en la paleta de color.
La curva de la loss apoya esta observacion, y es por esto que **muchas pruebas las hice solo con 50 iteraciones.**\

<img src="Resultados/Losses/001-LossIter.png">\
En otras pruebas fue interesante el cambio en la ieracion 100, esos archivos estan en la carpeta de resultado, aunque no estan referenciados en este archivo.

**Ademas fueron removidas del proyecto las imagenes de los lotes con configuracion de parametros diferentes de los que elegi comentar en la pregunta 8**

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:\
En la seiguiente imagen se muestran los resultados de algunas variaciones de parametros, las imagenes son tomadas de la iteracion numero 20\
<img src="Resultados/Iter20ComparacionParametros.png">\
La imagen 1 usa los pesos propuestos en el trabajo
```
total_variation_weight = 0.1
style_weight = 10
content_weight = 1
```
La imagen 2 se genero modificando el peso del estilo a **`style_weight = 100`** y podemos notar el incremento de la extura por ejemplo en la pared a la derecha del rio, se nota un remolino amarillo que no esta tan presente en la imagen 1.\
La imagen 3 es muy interesante, en esta se configuro **`total_variation_weight = 50`**. Esto penaliso los cambios en pixels cercanos y la imgen resultante es menos definida, las lineas perdieron nitides y se combinaron colores.\
La imagen 4 se genero usando **`content_weight = 50`** a primera vista parece muy similar a la numero 1, pero se puede notar que respeta mas el dibujo de la fotografia en la precencia de mayor detalle en los reflejos de los edificios en el rio y los frentes de los edificios. 


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:\
Se adjunta una muestra aqui junto al codigo y otras imagenes estan disponibles en la carpeta Resultados.\
<img src="Resultados/TestColorPersonaMala.png">